Installing libraries

In [6]:
import stanza
import numpy as np
import pandas as pd

In [71]:
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
import os
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

import re
# import pickle
from emot.emo_unicode import UNICODE_EMOJI # For emojis
from emot.emo_unicode import EMOTICONS_EMO # For EMOTICONS

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/venkatasaisumanthsadu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/venkatasaisumanthsadu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/venkatasaisumanthsadu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/venkatasaisumanthsadu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Importing bad words vocab and dataset

In [7]:
bad_words=[]
with open('./data/bad-words.csv') as f:
    for line in f.readlines():
        bad_words.append(line.split('\n')[0])

bad_words_set = set(bad_words)
dataset = pd.read_csv('news_comments.csv')

In [70]:
dataset

,Unnamed: 0,comments
0,0,Man this whole rivalry between Michigan and Oh...
1,1,I get this feeling that derailments happen A L...
2,2,They happen a lot. I work in shipping and use ...
3,3,Weird that when we don't invest in infrastruct...
4,4,"Well, they've clearly learned from East Palest..."
...,...,...
51698,51698,Ukraine also has the average of superior train...
51699,51699,Does NK make their own shells? Or was Russia b...
51700,51700,counterpoint: if there's one thing Best Korea ...
51701,51701,American intelligence is absolutely crucial to...


In [73]:
def remove_urls(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    return (text)

# def remove_contractions(text) :
#     expanded_words = []
#     for word in text.split():
#         expanded_words.append(contractions.fix(word))
#         expanded_text = ' '.join(expanded_words)
#     return expanded_text


remove_non_english = lambda s: re.sub(r'[^a-zA-z]', ' ', s)
remove_spaces = lambda s: re.sub(' +',' ', s)


In [76]:

def cleaning(text):
    #remove urls
    text = remove_urls(text)
    #remove html tags
    text = BeautifulSoup(text, "lxml").text
    #remove contractions 
    # text = remove_contractions(text)
    #remove non-alphabetic chars 
    text = remove_non_english(text)
    #lowercase
    text = text.lower( )
    #remove extra spaces 
    text = remove_spaces(text)
    
    return text

In [77]:
dataset['cleaned_comments'] = list(map(cleaning, dataset.comments))

def convert_emojis(text):
    for emot in UNICODE_EMOJI:
        text = text.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
    return text

dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda row: convert_emojis(str(row)))

In [ ]:
# from nltk.corpus import stopwords

# to_remove = ['not']
# new_stopwords = set(stopwords.words('english')).difference(to_remove)

# dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda x: " ".join(x for x in x.split() if x not in new_stopwords))

# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()

# dataset['cleaned_comments'] = dataset['cleaned_comments'].apply(lambda x: " ".join([lemmatizer.lemmatize(word) for word in x.split()]))


Importing Stanza POS tagger

In [79]:
pos_tagger = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos',use_gpu=True)

2023-03-20 23:55:23 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-03-20 23:55:24 WARNING: Can not find mwt: default from official model list. Ignoring it.
2023-03-20 23:55:24 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

2023-03-20 23:55:24 WARNING: GPU requested, but is not available!
2023-03-20 23:55:24 INFO: Using device: cpu
2023-03-20 23:55:24 INFO: Loading: tokenize
2023-03-20 23:55:24 INFO: Loading: pos
2023-03-20 23:55:25 INFO: Done loading processors!


POS Tags for our dataset

In [87]:
dataset_sample = dataset[:50].copy()

In [101]:
dataset_sample['cleaned_comments'][1]

'i get this feeling that derailments happen a lot and we re only paying attention now because of what happened in ohio that said why the fuck do they happen so often '

In [91]:
pos_tags_dataset_sample =[]
for sentence in dataset_sample.iloc[:,2].values.tolist():
    doc = pos_tagger(sentence)
    pos_tags_dataset_sample.append([word.xpos for sent in doc.sentences for word in sent.words])

Writing the tags to a txt file (Simply)

In [92]:
with open('pos_tags_dataset_sample.txt','w') as f:
    f.writelines([str(i) for i in pos_tags_dataset_sample])

In [93]:
# pos_tags_dataset_sample=[]
# with open('/Users/venkatasaisumanthsadu/Documents/clg-documents/CSCI544/project/CSCI544_NLP_Project-main/pos_tags_dataset_sample.txt','r') as f:
#     pos_tags_dataset_sample.append(f.readlines())

Labelling the dataset

In [140]:
with open('pos_tags_dataset_sample.txt','r') as f:
    Lines = f.readlines()

    # print([tag.strip("'") for tag in Lines[2].strip('\n').split(', ')])
    pos_tags_dataset_sample = []
    for i in range(len(Lines)):
        pos_tags_dataset_sample.append([tag.strip("'") for tag in Lines[i].strip('\n').split(', ')])

In [47]:
len(dataset['comments'][0].split(' '))

13

In [68]:
len(dataset['comments'][1][:].split(' '))

31

In [141]:
pos_tags_dataset_sample[2]

['PRP',
 'VBP',
 'DT',
 'NN',
 'PRP',
 'VBP',
 'IN',
 'NN',
 'CC',
 'VBP',
 'JJ',
 'NN',
 'NNS',
 'IN',
 'NN',
 'NN',
 'RB',
 'CC',
 'PRP',
 'VBZ',
 'IN',
 'DT',
 'JJ',
 'NN',
 'PRP',
 'VBP',
 'PRP',
 'PRP',
 'VBP',
 'JJ',
 'VBG',
 'IN',
 'DT',
 'NN',
 'NN']

In [142]:
pos_tags_dataset_sample[1]

['PRP',
 'VBP',
 'DT',
 'NN',
 'IN',
 'NNS',
 'VBP',
 'DT',
 'NN',
 'CC',
 'PRP',
 'VBP',
 'RB',
 'VBG',
 'NN',
 'RB',
 'IN',
 'IN',
 'WP',
 'VBD',
 'IN',
 'NNP',
 'WDT',
 'VBD',
 'WRB',
 'DT',
 'NN',
 'VBP',
 'PRP',
 'VB',
 'RB',
 'RB']

In [129]:
len(dataset_sample['cleaned_comments'][2].split())

35

In [119]:
dataset_sample['cleaned_comments'][0]

'man this whole rivalry between michigan and ohio is getting out of hand '

In [144]:
labels= []
word_indices=[]
for i,sentence in enumerate(dataset_sample['cleaned_comments']):
    #capture bw indices for each sentence
    flag = False
    for j,word in enumerate(sentence.split()):
        if word.lower() in bad_words_set:
            flag = True
            pos_tags_dataset_sample[i][j] = 'BW'
            print(i,j)
            continue
    labels.append(1) if flag else labels.append(0)

1 26
7 10
7 16
15 31
16 12
16 48
20 26
23 25
25 58
29 11
34 35
34 43
34 52
34 64
35 10
36 4
39 18
40 9
44 0
45 2
47 9


Percentage of bad sentences

In [5]:
np.count_nonzero(labels)

14439

In [6]:
np.count_nonzero(labels)/len(labels)

0.27926812757480224

Separating good and bad sentences

In [7]:
dataset['labels'] = labels

In [8]:
good_sentences = dataset[dataset['labels']==0]
bad_sentences = dataset[dataset['labels']==1]

In [12]:
len(good_sentences)
good_sentences.to_csv('good_sentences.csv')

In [13]:
len(bad_sentences)
bad_sentences.to_csv('bad_sentences.csv')

In [21]:
sample_bad = bad_sentences.sample(10)

In [23]:
sample_bad

,Unnamed: 0,comments,labels
33865,33865,Basically what I'm saying is that if they clai...,1
15899,15899,The Ferenghi do have a general reservation aga...,1
24288,24288,PPP Fraud helped business owners buy all their...,1
50279,50279,These are a great alternative to abandoning yo...,1
16376,16376,No vinyl chloride or pre-product has been dete...,1
33306,33306,"What the fuck, you are regularly finding dead ...",1
24605,24605,"*If* he's being sarcastic, fascists managed to...",1
44891,44891,It'd be terrifying as fuck when you awake and ...,1
12769,12769,> Gaetz’s father is famous for fixing things f...,1
46345,46345,Union Carbide killed over 3700 people and inju...,1


Lucene

In [ ]:
# import lucene
# from java.io import StringReader
# from org.apache.lucene.analysis.standard import StandardAnalyzer
# from org.apache.lucene.index import DirectoryReader
# from org.apache.lucene.search.similarities import ClassicSimilarity
# from org.apache.lucene.search import IndexSearcher
# from org.apache.lucene.queryparser.classic import QueryParser
# from org.apache.lucene.store import SimpleFSDirectory
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # Initialize Lucene
# lucene.initVM()

# # Set up the analyzer and similarity algorithm
# analyzer = StandardAnalyzer()
# similarity = ClassicSimilarity()

# # Set up the index
# index_dir = SimpleFSDirectory(File("index"))
# searcher = IndexSearcher(DirectoryReader.open(index_dir))
# searcher.setSimilarity(similarity)

# # Define the query and tags
# query = "python programming"
# tags = ["python", "programming"]

# # Tokenize and vectorize the tags using TF-IDF
# tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
# tfidf_matrix = tfidf.fit_transform(tags)

# # Search the index for similar sentences
# query_parser = QueryParser("content", analyzer)
# query = query_parser.parse(query)
# top_docs = searcher.search(query, 10)
# for score_doc in top_docs.scoreDocs:
#     doc = searcher.doc(score_doc.doc)
#     sentence = doc.get("content")
#     tfidf_score = cosine_similarity(tfidf_matrix, tfidf.transform([sentence]))[0][0]
#     if tfidf_score > 0.5:
#         print(f"Similar sentence found: {sentence}")

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define the sentences
sentences = [
    # "The quick brown cat jumps over the lazy dog"
]

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Compute the tf-idf matrix
tfidf_matrix = vectorizer.fit_transform(sentences)

# Compute the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

# Print the similarity matrix
print(cosine_sim_matrix)


[[1.         0.25861529 0.25861529]
 [0.25861529 1.         0.25861529]
 [0.25861529 0.25861529 1.        ]]


In [4]:
# with open('pos_tags_dataset_duplicate.txt','r') as f:
#     Lines = f.readlines()

#     # print(Lines[0])
#     lst = []
#     for i in range(len(Lines)):
#         lst.append(list(Lines[i]))

'NN', 'DT', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNP', 'VBZ', 'VBG', 'IN', 'IN', 'NN', '.'

